In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('./open/train.csv')
test = pd.read_csv('./open/test.csv')

In [4]:
train = pd.read_csv('./open/train.csv')
train[(train['Age'] > 15) & (train['Employment_Status'] != 'Not Working')]

,ID,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,...,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status,Income
0,TRAIN_00000,63,M,Middle (7-8),Full-Time,4,Social Services,Services,White,All other,...,Native,US,US,US,Nonfiler,0,0,0,Unknown,425
1,TRAIN_00001,37,M,Associates degree (Vocational),Full-Time,52,Entertainment,Services,White,All other,...,Native,US,US,US,Single,0,0,0,Under Median,0
2,TRAIN_00002,58,F,High graduate,Full-Time,52,Manufacturing (Non-durable),Admin Support (include Clerical),Black,All other,...,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),3411,0,0,Under Median,860
3,TRAIN_00003,44,M,High graduate,Full-Time,52,Retail,Technicians & Support,White,All other,...,Native,US,US,US,Single,0,0,0,Under Median,850
4,TRAIN_00004,37,F,High graduate,Full-Time,52,Retail,Sales,White,All other,...,Native,US,US,US,Head of Household (HOH),0,0,0,Unknown,570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19993,TRAIN_19993,41,M,High graduate,Full-Time,52,Retail,Sales,White,All other,...,Native,US,US,US,Single,0,0,0,Under Median,900
19995,TRAIN_19995,33,M,High graduate,Children or Armed Forces,52,Manufacturing (Durable),Handlers/Cleaners,White,All other,...,Native,US,US,US,Single,0,0,0,Under Median,1300
19996,TRAIN_19996,20,F,College,Full-Time,12,Education,Admin Support (include Clerical),White,Mexican-American,...,Native,US,Mexico,Mexico,Nonfiler,0,0,0,Under Median,850
19997,TRAIN_19997,22,M,College,Children or Armed Forces,52,Transportation,Technicians & Support,White,All other,...,Native,US,US,US,Single,0,0,0,Unknown,999


In [5]:
tp = train[['Education_Status','Age','Income']]
tp[tp['Age'] > 15].groupby('Education_Status').mean()

,Age,Income
Education_Status,,
Associates degree (Academic),38.141215,782.740558
Associates degree (Vocational),39.929825,867.269906
Bachelors degree,39.136503,699.367076
College,36.234581,651.468108
Doctorate degree,48.551724,575.528736
Elementary (1-4),49.186667,343.440000
Elementary (5-6),45.649842,325.116719
High Freshman,37.182952,405.316008
High Junior,32.793651,483.623932


In [6]:
len(train[train['Income'] == 0])/len(train)

0.43485

In [7]:
len(train[(train['Income'] == 0) & (train['Age'] > 15) & (train['Employment_Status'] != 'Not Working')])/len(train)

0.2606

In [ ]:
from pycaret.regression import *

train = pd.read_csv('./open/train.csv')
test = pd.read_csv('./open/test.csv')

# object_columns = train.select_dtypes(include=['object']).columns
# for i in object_columns:
#     train[i] = train[i].astype('category')
#     test[i] = test[i].astype('category')

data = train[(train['Age'] > 15) & (train['Employment_Status'] != 'Not Working') & (train['Income'] < 3000)]
data = data.drop('ID',axis=1)
# PyCaret 회귀 모델 설정
regression_setup = setup(data, target='Income',
                         max_encoding_ohe=5,
                         train_size=0.9,
                         fold=10,
                         session_id=42)

# 다양한 회귀 모델 비교
best_model = compare_models(n_select=3, sort='RMSE')

,Description,Value
0,Session id,42
1,Target,Income
2,Target type,Regression
3,Original data shape,"(16357, 22)"
4,Transformed data shape,"(16357, 32)"
5,Transformed train set shape,"(14721, 32)"
6,Transformed test set shape,"(1636, 32)"
7,Ordinal features,1
8,Numeric features,5
9,Categorical features,16


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,01:45:39
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)


Processing:   0%|          | 0/87 [00:00<?, ?it/s]

In [5]:
import pandas as pd
from sklearn.model_selection import KFold
import optuna
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

train = pd.read_csv('./open/train.csv').drop('ID',axis=1)
test = pd.read_csv('./open/test.csv').drop('ID',axis=1)
object_columns = train.select_dtypes(include=['object']).columns

for i in object_columns:
    train[i] = train[i].astype('category')
    test[i] = test[i].astype('category')
    
data = train[(train['Age'] > 15) & (train['Income'] < 3000) & (train['Employment_Status'] == 'Not Working')]
X = data.drop('Income',axis=1)
y = data[['Income']]
# 교차 검증을 위한 KFold 객체 생성

kf = KFold(n_splits=10, shuffle=True, random_state=42)

def objective(trial):
    # 하이퍼파라미터 검색 범위 지정
    params = {
        'objective': 'rmse',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100)
    }

    # 교차 검증을 통한 성능 측정
    rmses = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # LightGBM 모델 정의
        model = lgb.LGBMRegressor(**params)

        # 모델 훈련
        model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=100, verbose=False)

        # 검증 데이터로 RMSE 계산
        y_pred = model.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        rmses.append(rmse)

    return sum(rmses) / len(rmses)  # 교차 검증 평균 RMSE 반환

# Optuna 최적화 실행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# 최적 하이퍼파라미터 출력
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


[I 2024-03-27 12:43:12,106] A new study created in memory with name: no-name-91a918d7-ffc7-496d-af25-9a156441cca1
C:\Users\User\AppData\Local\Temp\ipykernel_13184\2965203895.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
C:\Users\User\AppData\Local\Temp\ipykernel_13184\2965203895.py:30: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
C:\Users\User\AppData\Local\Temp\ipykernel_13184\2965203895.py:32: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/r

[LightGBM] [Warning] feature_fraction is set=0.46873228379207216, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.46873228379207216
[LightGBM] [Warning] bagging_fraction is set=0.9995050862444139, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9995050862444139
[LightGBM] [Warning] lambda_l1 is set=0.001961129273181537, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.001961129273181537
[LightGBM] [Warning] lambda_l2 is set=0.0003529349088970434, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0003529349088970434
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.46873228379207216, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.46873228379207216
[LightGBM] [Warning] bagging_fraction is set=0.9995050862444139, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9995050862444139
[LightGBM

C:\Users\User\anaconda3\envs\project\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\User\anaconda3\envs\project\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\User\anaconda3\envs\project\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\User\anaconda3\envs\project\lib\site-packages\lightgb

[LightGBM] [Warning] feature_fraction is set=0.46873228379207216, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.46873228379207216
[LightGBM] [Warning] bagging_fraction is set=0.9995050862444139, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9995050862444139
[LightGBM] [Warning] lambda_l1 is set=0.001961129273181537, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.001961129273181537
[LightGBM] [Warning] lambda_l2 is set=0.0003529349088970434, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0003529349088970434
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.46873228379207216, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.46873228379207216
[LightGBM] [Warning] bagging_fraction is set=0.9995050862444139, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9995050862444139
[LightGBM

C:\Users\User\anaconda3\envs\project\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\User\anaconda3\envs\project\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\User\anaconda3\envs\project\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\User\anaconda3\envs\project\lib\site-packages\lightgb

...

[LightGBM] [Warning] feature_fraction is set=0.827497113143864, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.827497113143864
[LightGBM] [Warning] bagging_fraction is set=0.7236441531882767, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7236441531882767
[LightGBM] [Warning] lambda_l1 is set=0.0005696332693168291, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0005696332693168291
[LightGBM] [Warning] lambda_l2 is set=1.8237212910955652e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.8237212910955652e-07
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.827497113143864, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.827497113143864
[LightGBM] [Warning] bagging_fraction is set=0.7236441531882767, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7236441531882767
[LightGBM] [W

C:\Users\User\anaconda3\envs\project\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\User\anaconda3\envs\project\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')
C:\Users\User\anaconda3\envs\project\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\User\anaconda3\envs\project\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback 

[LightGBM] [Warning] feature_fraction is set=0.827497113143864, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.827497113143864
[LightGBM] [Warning] bagging_fraction is set=0.7236441531882767, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7236441531882767
[LightGBM] [Warning] lambda_l1 is set=0.0005696332693168291, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0005696332693168291
[LightGBM] [Warning] lambda_l2 is set=1.8237212910955652e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.8237212910955652e-07
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.827497113143864, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.827497113143864
[LightGBM] [Warning] bagging_fraction is set=0.7236441531882767, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7236441531882767
[LightGBM] [W

C:\Users\User\anaconda3\envs\project\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\User\anaconda3\envs\project\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\User\anaconda3\envs\project\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\User\anaconda3\envs\project\lib\site-packages\lightgb

[LightGBM] [Warning] feature_fraction is set=0.827497113143864, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.827497113143864
[LightGBM] [Warning] bagging_fraction is set=0.7236441531882767, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7236441531882767
[LightGBM] [Warning] lambda_l1 is set=0.0005696332693168291, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0005696332693168291
[LightGBM] [Warning] lambda_l2 is set=1.8237212910955652e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.8237212910955652e-07
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.827497113143864, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.827497113143864
[LightGBM] [Warning] bagging_fraction is set=0.7236441531882767, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7236441531882767
[LightGBM] [W

In [6]:
study.best_params

{'lambda_l1': 0.001961129273181537,
 'lambda_l2': 0.0003529349088970434,
 'num_leaves': 224,
 'learning_rate': 0.050922001997949315,
 'feature_fraction': 0.46873228379207216,
 'bagging_fraction': 0.9995050862444139,
 'bagging_freq': 5,
 'min_child_samples': 30}

In [5]:
lgb_model = lgb.LGBMRegressor(**study.best_params)
lgb_model.fit(X,y)

[LightGBM] [Warning] feature_fraction is set=0.8056873190486198, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8056873190486198
[LightGBM] [Warning] bagging_fraction is set=0.940402467219656, subsample=1.0 will be ignored. Current value: bagging_fraction=0.940402467219656
[LightGBM] [Warning] lambda_l1 is set=1.0413624031262543e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.0413624031262543e-06
[LightGBM] [Warning] lambda_l2 is set=8.902854507433748e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.902854507433748e-07
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7


LGBMRegressor(bagging_fraction=0.940402467219656, bagging_freq=7,
              feature_fraction=0.8056873190486198,
              lambda_l1=1.0413624031262543e-06, lambda_l2=8.902854507433748e-07,
              learning_rate=0.06463504980900336, min_child_samples=70,
              num_leaves=244)

In [8]:
import pandas as pd
from sklearn.model_selection import KFold
import optuna
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

train = pd.read_csv('./open/train.csv').drop('ID',axis=1)
test = pd.read_csv('./open/test.csv').drop('ID',axis=1)
object_columns = train.select_dtypes(include=['object']).columns

for i in object_columns:
    train[i] = train[i].astype('category')
    test[i] = test[i].astype('category')
    
data = train[(train['Age'] > 15) & (train['Income'] < 3000)]
X = data.drop('Income',axis=1)
y = data[['Income']]
# 교차 검증을 위한 KFold 객체 생성

kf = KFold(n_splits=10, shuffle=True, random_state=42)

def objective(trial):
    # 하이퍼파라미터 검색 범위 지정
    params = {
        'objective': 'RMSE',
        'iterations': 2000,
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
        'random_strength': trial.suggest_loguniform('random_strength', 1e-3, 10.0),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.1, 1.0),
        'border_count': trial.suggest_int('border_count', 1, 255),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide'])
    }

    # 교차 검증을 통한 성능 측정
    rmses = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # CatBoost 모델 정의
        model = CatBoostRegressor(**params, silent=True, cat_features=list(object_columns))

        # 모델 훈련
        model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=100, verbose=False)

        # 검증 데이터로 RMSE 계산
        y_pred = model.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        rmses.append(rmse)

    return sum(rmses) / len(rmses)  # 교차 검증 평균 RMSE 반환

# Optuna 최적화 실행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

# 최적 하이퍼파라미터 출력
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


[I 2024-03-25 14:50:34,412] A new study created in memory with name: no-name-efb295eb-4743-4b57-acb4-bddacd12ae6d
C:\Users\User\AppData\Local\Temp\ipykernel_15016\564935787.py:27: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
C:\Users\User\AppData\Local\Temp\ipykernel_15016\564935787.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
C:\Users\User\AppData\Local\Temp\ipykernel_15016\564935787.py:30: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna

[I 2024-03-25 17:26:24,547] Trial 4 finished with value: 492.61912660354994 and parameters: {'learning_rate': 0.0012113211792967274, 'depth': 7, 'l2_leaf_reg': 0.002802935070269858, 'random_strength': 0.018266265434790475, 'bagging_temperature': 0.8245636931504466, 'border_count': 171, 'grow_policy': 'SymmetricTree'}. Best is trial 3 with value: 484.9310884845253.
C:\Users\User\AppData\Local\Temp\ipykernel_15016\564935787.py:27: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
C:\Users\User\AppData\Local\Temp\ipykernel_15016\564935787.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': 

[I 2024-03-25 21:26:11,733] Trial 9 finished with value: 487.2640562059563 and parameters: {'learning_rate': 0.0236814381032317, 'depth': 6, 'l2_leaf_reg': 0.8574640234941264, 'random_strength': 0.35810681192523425, 'bagging_temperature': 0.13987572099132195, 'border_count': 11, 'grow_policy': 'Lossguide'}. Best is trial 8 with value: 484.493325144232.


Number of finished trials:  10
Best trial:
  Value:  484.493325144232
  Params: 
    learning_rate: 0.005746067154098378
    depth: 10
    l2_leaf_reg: 0.22009313587635276
    random_strength: 0.45687561571445706
    bagging_temperature: 0.6606541325502003
    border_count: 106
    grow_policy: SymmetricTree


In [92]:
pred = 0.7*lgb_model.predict(test.drop('ID',axis=1)) + 0.3*cat_model.predict(test.drop('ID',axis=1))
for i in test[(test['Age'] < 15) | (test['Employment_Status'] == 'Not Working')].index:
    pred[i] = 0
    
for i in range(len(pred)):
    if pred[i] < 0:
        pred[i] = 0
pred

array([  0.        ,   0.        , 414.52398498, ..., 365.06567722,
         0.        , 528.04044127])

In [95]:
len([i for i in pred if i == 0])

1954

In [71]:
sub = pd.read_csv('./open/sample_submission.csv', index_col=0)
sub['Income'] = pred
sub.to_csv('./lgb.csv')

In [80]:
# test = pd.read_csv('./open/test.csv')
# tp = pd.read_csv('./lgb.csv',index_col=0)
# pred = tp['Income']
# print(np.mean(pred))
# for i in test[test['Employment_Status'] == 'Not Working'].index:
#     pred[i] = 0
# print(np.mean(pred))
# sub = pd.read_csv('./open/sample_submission.csv', index_col=0)
# sub['Income'] = pred
# sub.to_csv('./lgb-nw.csv')

532.8826780873919
531.3588360937576
